### Amazon Cookbooks Topic-modeling: Create Categories and Scarpe for Pics

__Purpose:__
The purpose of this notebook is to:
1. Create classes for classification task
2. Calc average ratings per book for regresson
3. Code for scraping of book images   

In [98]:
import numpy as np
import pandas as pd
import re, string

#import contractions
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from langdetect import detect
from textblob import TextBlob

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA, LatentDirichletAllocation, TruncatedSVD, NMF
from sklearn.cluster import KMeans

from joblib import dump, load
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline

    
# --- Parser for reading in the Amazon json files (can be used for both reviews and metadata)
# --- credits folloing parse() method to Julian McAuley UCSD: http://jmcauley.ucsd.edu/data/amazon/ 
def parse(path):
    g = open(path, 'r')
    for l in g:
        yield json.loads(l)

#### __Read Cookbook Reviews and Cookbook Metadata__

In [13]:
# --- Read cookbooks tables:  Products from meta_cookbooks.json and reviews from Cookbooks.json
ckbk_meta_file = "../data/amzn/meta_Cookbooks.json"
ckbk_reviews_file = "../data/amzn/Cookbooks.json"

ckbks = pd.read_json(ckbk_meta_file, lines=True)
print(ckbks.columns)
ckbks.drop(['also_buy', 'also_buy', 'brand', 'rank', 'also_view', 'main_cat', 'price', 'image', 'feature', 'date',
       'similar_item', 'details'], axis=1, inplace=True)
ckbks

Index(['category', 'description', 'title', 'also_buy', 'brand', 'rank',
       'also_view', 'main_cat', 'price', 'asin', 'image', 'feature', 'date',
       'similar_item', 'details'],
      dtype='object')


,category,description,title,asin
0,"[Books, Cookbooks, Food & Wine, Regional & Int...",[Though at first glance too luscious for the p...,France: The Beautiful Cookbook- Authentic Reci...,0002154129
1,"[Books, Cookbooks, Food & Wine, Regional & Int...",[Lorenza de Medici has published more than 30 ...,"Italy, The Beautiful Cookbook: Authentic Recip...",0002154463
2,"[Books, Cookbooks, Food & Wine, Regional & Int...",[Two collections of California's best. The lat...,"California, The Beautiful Cookbook: Authentic ...",0002159317
3,"[Books, Cookbooks, Food & Wine, Regional & Int...",[Fifth in the series that includes the best-se...,America: The Beautiful Cookbook,000215854X
4,"[Books, Cookbooks, Food & Wine, Regional & Int...",[The latest addition to this popular series is...,Mexico The Beautiful Cookbook: Authentic Recip...,000215949X
...,...,...,...,...
44831,"[Books, Cookbooks, Food & Wine, Special Diet]",[Recipe Excerpts from The Joy Fit Club Ch...,"Joy Fit Club: Cookbook, Diet Plan & Inspiration",B01FMW2MZ6
44832,"[Books, Cookbooks, Food & Wine, Beverages & Wine]","[""A fascinating look at the place and function...",Drinking in America: Our Secret History,B01FMVXYF4
44833,"[Books, Cookbooks, Food & Wine]",[ALL NEW 5th Edition to kick off 2017 with exp...,Entertaining with Vegetables: A Recipe Collect...,B01FUPYP0A
44834,"[Books, Cookbooks, Food & Wine]",[Grit Country Series Magazine 2016 (Volume 10...,Grit Guide to Cast Iron Cooking Magazine 2016,B01FV1VJ66


#### Create Categories
1. Cookbooks on Amazon are under the following categories: Books > Cookbooks, Food & Wine 
   These are level 2 category.  Most of the books have sub-categories. Those that are in sparse sub-cookbook categpories are merged with their approximate neighbor branches. These are as follows:

Sparse Level 3 cateogires are merged with their neighbor brances such that there are no classes less than a 1000 left. The merging is done as follows:

"Professional Cooking": "Cooking Education & Reference"
"Celebrities & TV Shows": "Cooking Education & Reference"
"Canning & Preserving": "Cooking Methods"
"Desserts": "Baking"
"Outdoor Cooking": "Cooking Methods"
"Quick & Easy": "Cooking Methods"

   

In [14]:
#ckbks['level2cat'] = ckbks['category'].apply(lambda x: x[2])
#
ckbks['cats'] = ckbks['category'].apply(lambda x: x[2] if len(x)==3 else x[1])
print(f"Total number of categories: {ckbks['cats'].value_counts().sum()}")
ckbks['cats'].value_counts()

Total number of categories: 44836


Regional & International         11597
Cookbooks, Food & Wine            4963
Cooking by Ingredient             4853
Beverages & Wine                  4569
Special Diet                      3902
Baking                            3097
Cooking Education & Reference     2252
Cooking Methods                   2038
Main Courses & Side Dishes        1441
Entertaining & Holidays           1339
Kitchen Appliances                1241
Vegetarian & Vegan                1013
Quick & Easy                       784
Outdoor Cooking                    748
Desserts                           574
Canning & Preserving               299
Celebrities & TV Shows              84
Professional Cooking                42
Name: cats, dtype: int64

In [15]:
switch_categories = {"Professional Cooking": "Cooking Education & Reference",
                    "Celebrities & TV Shows": "Cooking Education & Reference",
                    "Canning & Preserving": "Cooking Methods",
                    "Desserts": "Baking",
                    "Outdoor Cooking": "Cooking Methods",
                    "Quick & Easy": "Cooking Methods",
                   }

ckbks['new_category'] = ckbks['cats'].apply(lambda x: switch_categories[x] if x in switch_categories.keys() else x)
print(f"Total number of books: {ckbks['new_category'].value_counts().sum()}")
list(ckbks['new_category'].value_counts().index)

Total number of books: 44836


['Regional & International',
 'Cookbooks, Food & Wine',
 'Cooking by Ingredient',
 'Beverages & Wine',
 'Special Diet',
 'Cooking Methods',
 'Baking',
 'Cooking Education & Reference',
 'Main Courses & Side Dishes',
 'Entertaining & Holidays',
 'Kitchen Appliances',
 'Vegetarian & Vegan']

#### Calclate Average Book Ratings
1. Cookbooks on Amazon are under the following categories: Books > Cookbooks, Food & Wine 
   These are level 2 category.  Most of the books have sub-categories. Those that are in sparse sub-cookbook categpories are merged with their approximate neighbor branches. These are as follows:



In [5]:
#ckbk_reviews_file
from collections import defaultdict
book_ratings = {}

for review in parse(ckbk_reviews_file):
    try:
        book_ratings[review['asin']]['ratings'].append(review['overall'])
    except:
        book_ratings[review['asin']] = {'ratings': [review['overall']]}

#print(sum(ratings) / len(ratings))
#print(ratings)

for asin in book_ratings.keys():
    book_ratings[asin]['total_ratings'] = len(book_ratings[asin]['ratings']) 
    book_ratings[asin]['avg_rating'] = np.array(book_ratings[asin]['ratings']).sum()/book_ratings[asin]['total_ratings']

In [10]:
print(len(book_ratings.keys()))
#dump(book_ratings, "../model/cookbook_ratings.joblib")

output_file = "../data/amzn/cookbook_ratings.csv"
open(output_file, "w").write(json.dumps(book_ratings))

15782


3363236

In [57]:
#print(book_ratings['1101903910']['avg_rating'])
ckbks["ratings"] = ckbks['asin'].apply(lambda x: book_ratings[x]['avg_rating'] if x in book_ratings.keys() else 0)

In [68]:
#gpby = ckbks.groupby('new_category')
#specaiDt=gpby.get_group('Special Diet')
new_df = ckbks.sort_values(["ratings"], ascending=False).groupby("new_category").head(5)#.apply(lambda x: x.sort_values(['ratings']))
asins = new_df['asin']
print(asins)

11179    0801556104
6098     0618048456
12853    0847837955
31310    1578601657
31299    157833523X
12797    0847819248
6029     0615863418
1407     0151003106
37528    1844831868
12772    0841603561
31282    1578264316
12749    0836199456
24797    1493762745
31260    1577491076
24796    1493762885
31248    1577192761
6149     0620682760
31212    1574865277
31200    1574861204
37701    184597378X
41506    1945185023
31160    1573241156
6220     0670841420
6253     0670880221
12489    0816510210
41440    1943366012
12861    0847838390
12884    0847843378
5897     061556920X
13062    0848731565
5924     0615636608
12998    0848724496
21755    1441123970
21763    1441310185
12950    0848706773
5993     0615776469
21801    1442102306
37772    1845979192
13223    0848744381
12083    0811844048
37962    1849496609
6262     0671043455
41333    1941303587
30962    1571459987
41408    1942613032
12369    0812902130
6290     0671438638
6295     067144932X
44517    B00682TZ8M
25266    1500195502


In [90]:
import requests
import re
from bs4 import BeautifulSoup

amzn_link = "https://www.amazon.com/s?k=xxxxx&ref=nb_sb_noss"

urls= [amzn_link.replace("xxxxx", asin) for asin in asins]
test_url=amzn_link.replace("xxxxx", "0192729578")
#print(urls)

response = requests.get(test_url)
#if response.status_quote>= 200:
soup = BeautifulSoup(response.text, 'html')
print(soup.prettify())
soup.find_all('img')

<!DOCTYPE html>
<!--[if lt IE 7]> <html lang="en-us" class="a-no-js a-lt-ie9 a-lt-ie8 a-lt-ie7"> <![endif]-->
<!--[if IE 7]>    <html lang="en-us" class="a-no-js a-lt-ie9 a-lt-ie8"> <![endif]-->
<!--[if IE 8]>    <html lang="en-us" class="a-no-js a-lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!-->
<html class="a-no-js" lang="en-us">
 <!--<![endif]-->
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="content-type"/>
  <meta charset="utf-8"/>
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <title dir="ltr">
   Robot Check
  </title>
  <meta content="width=device-width" name="viewport"/>
  <link href="https://images-na.ssl-images-amazon.com/images/G/01/AUIClients/AmazonUI-3c913031596ca78a3768f4e934b1cc02ce238101.secure.min._V1_.css" rel="stylesheet"/>
  <script>
   if (true === true) {
    var ue_t0 = (+ new Date()),
        ue_csm = window,
        ue = { t0: ue_t0, d: function() { return (+new Date() - ue_t0); } },
        ue_furl = "fls-na.amazon.com",
     

[<img src="https://images-na.ssl-images-amazon.com/captcha/druexhzz/Captcha_saatxqfpse.jpg"/>,
 <img src="https://fls-na.amazon.com/1/oc-csi/1/OP/requestId=DS36Y3FM4WJEZ6CFGJB6&amp;js=0"/>]

In [91]:
imgUrl_list={
    '1328631346':'https://m.media-amazon.com/images/I/91+4AszMZFL._AC_UY654_QL65_ML3_.jpg',
    '1727408403':'https://m.media-amazon.com/images/I/51I+OG-KypL._AC_UY500_QL65_ML3_.jpg',
    '1723830666':'https://m.media-amazon.com/images/I/51+4Oe0IyWL._AC_UY500_QL65_ML3_.jpg'
}

In [96]:
import urllib.request as ul



for key, value in imgUrl_list.items():
    filedata = ul.urlopen(value)
    datatowrite = filedata.read()
 
    with open('../data/amzn/'+key+".jpg", 'wb') as f:
        f.write(datatowrite)


In [118]:
def parse2(path):
    g = open(path, 'r')
    for l in g:
        yield eval(l)

meta2014file = "../data/amzn/2014Meta/metadata.json"
bookasins = list(ckbks['asin'])
bookurls = {}

for line in parse2(meta2014file):
    prod_keys = line.keys()
    if 'asin' in prod_keys and 'imUrl' in prod_keys:
        if line['asin'] in bookasins:
            bookurls[line['asin']] = line['imUrl']

print(f"Completed gathering urls for cookbooks. Total urls gathered: {len(bookurls)}")

KeyboardInterrupt: 

In [120]:
print((bookurls))



{'0002154129': 'http://ecx.images-amazon.com/images/I/51ED0F88FXL.jpg', '000215949X': 'http://ecx.images-amazon.com/images/I/5159ZN5BGAL.jpg', '000215854X': 'http://ecx.images-amazon.com/images/I/21DVRx7iZ0L.jpg', '0002159317': 'http://ecx.images-amazon.com/images/I/51762E1Z2YL.jpg', '0002154463': 'http://ecx.images-amazon.com/images/I/51QNHEJ290L.jpg', '0002250713': 'http://ecx.images-amazon.com/images/I/51VQVZT596L.jpg', '0002251965': 'http://ecx.images-amazon.com/images/I/6145ZB15N8L.jpg', '0002252104': 'http://ecx.images-amazon.com/images/I/51TZC77XC5L.jpg', '0002251337': 'http://ecx.images-amazon.com/images/I/51EQN2CM4VL.jpg', '0002252015': 'http://ecx.images-amazon.com/images/I/61TMVT6MS9L.jpg', '0002251507': 'http://ecx.images-amazon.com/images/I/41lFbOz6sDL.jpg', '0002250519': 'http://ecx.images-amazon.com/images/I/51HBF54YANL.jpg', '0002250616': 'http://ecx.images-amazon.com/images/I/61AKEMNP19L.jpg', '0002250535': 'http://ecx.images-amazon.com/images/I/51QQG9CJQJL.jpg', '0002